In [ ]:
import matplotlib.pyplot as plt
import numpy as np

RUNS = 100_000
# avg between 48.2 and 48.8
BURDEN_BABIES = 48.5
CELLS = 100_000
# mitchell inference (calibration)
BURDEN_PER_YEAR = 14
BURDEN_PER_DIVISION = 1.2
ABC_4D = False  # infer tau from abc (4d abc)

In [ ]:
def compute_m_exp(cells: int) -> float:
    # a 2 is missing because in rust we divide by 2
    return 48.5 / (np.log((cells + 1)/2))

def compute_m_background(tau: float) -> float:
    # 2 because we assume no asymmetric divisions
    return BURDEN_PER_YEAR - 2 * tau * BURDEN_PER_DIVISION

In [ ]:
rng = np.random.default_rng(seed=42)

if ABC_4D:
    taus = rng.uniform(0.3, 5, RUNS)
else:
    taus = np.ones(RUNS, dtype=int)
plt.hist(taus, label="tau", bins=100)
plt.legend()
plt.show()

mu0s = rng.uniform(0.1, 20, RUNS)
plt.hist(mu0s, label="mu", bins=100)
plt.legend()
plt.show()

s_means = rng.uniform(0.01, 0.4, RUNS)
plt.hist(s_means, label="mean", bins=100)
plt.legend()
plt.show()

s_stds = rng.uniform(0.005, 0.1, RUNS)
plt.hist(s_stds, label="std", bins=100)
plt.legend()
plt.show()

plt.hist([compute_m_background(tau) for tau in taus], label="mu background", bins=100)
plt.legend()
plt.show()

In [ ]:
plt.hist(np.column_stack((taus, s_stds))[:, 1])

In [ ]:
with open("parameters.txt", "w") as f:
    for (tau, mu, mean, std) in np.column_stack((taus, mu0s, s_means, s_stds)):
        f.write(
            f"-c {CELLS} -y 82 -r 1 --tau {tau} --mu0 {mu} --mu-exp {compute_m_exp(CELLS)} --mu-division 1.2 --mu-background {compute_m_background(tau)} --mean-std {mean} {std}\n"
        )

In [ ]:
%%bash
head parameters.txt

In [ ]:
%%bash --out runs
wc -l parameters.txt

In [ ]:
runs = int(runs.split(" ")[0])
assert RUNS == runs, f"{RUNS} vs {runs}"